<a href="https://colab.research.google.com/github/Najme-naseri/Image-Procssing/blob/main/ImageSqueeze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import math
import shutil
from random import sample
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import cv2
import pdb
import time
from PIL import Image

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

from collections import OrderedDict

In [15]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive"

In [16]:
!pip install -q kaggle

In [17]:
!kaggle datasets download -d alxmamaev/flowers-recognition --unzip

 93% 209M/225M [00:01<00:00, 157MB/s]
100% 225M/225M [00:01<00:00, 140MB/s]


In [6]:
%ls

drive/  flowers/  sample_data/


In [18]:
# By defalt, set device to the CPU
deviceFlag = torch.device('cpu')

# Defalut is CPU, but as long as GPU is avaliable, then use GPU
if torch.cuda.is_available():
 
    print(f'Found {torch.cuda.device_count()} GPUs.')
    deviceFlag = torch.device('cuda:0') # Manually pick your cuda device. By default is 'cuda:0'

print(f'Now the deivce is set to {deviceFlag}')

Now the deivce is set to cpu


In [19]:
#DATA AUGMENTATION
#transformation for train data

training_transforms = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), # For GPU purpose
    # As we are going to do transfer learning with a ImageNet pretrained squeezeNet
    # so here we normalize the dataset being used here with the ImageNet stats
    # for better transfer learning performance
    transforms.Normalize([0.485, 0.456, 0.406], # RGB mean & std estied on ImageNet
                         [0.229, 0.224, 0.225])
])


In [20]:
#transformation for validation data
validation_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], # RGB mean & std estied on ImageNet
                         [0.229, 0.224, 0.225])
])

In [21]:
#transformation for test data
testing_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], # RGB mean & std estied on ImageNet
                         [0.229, 0.224, 0.225])
])

Data Split Function




create Train, Test & Valid directory

In [22]:
#get data directory path and return train, test & valid path in a dictionary
def split_data(flower_path):

  #create path for train, test & valid
    splited_data_path =  "/splited_data"
    data_path = {'train': os.path.join(splited_data_path, "train"), 
            'valid': os.path.join(splited_data_path, "valid"),
            'test':os.path.join(splited_data_path, "test")}

  #specify the ratio of data
    for phase in data_path.keys():
      if phase == 'train':
          ratio = 0.7
      elif phase == 'test':
          ratio = 0.2
      else:
          ratio = 0.1

      #for each train, test & valid phase we create a directory for each flower
      for name in os.listdir(flower_path):
        
        #create path for each flower
        data_directory_path = os.path.join(data_path[phase], name)

        #path of each flower data
        flower_data_path = os.path.join(flower_path, name)

        #make directory for each flower
        os.makedirs(data_directory_path, exist_ok=True)

        #Calculate the data size according to the phase
        #this part have problem 
        data_size = math.floor(ratio * len(os.listdir(flower_data_path)))

        #select random samples from data
        selected_samples = sample(os.listdir(flower_data_path), data_size)

        #for each sample, move data from flower directory
        #to created directories for train, test & valid
        #We use move so that we don't select duplicate data every time
        for img in selected_samples:
          shutil.move(os.path.join(flower_data_path, img), data_directory_path)


    return data_path

check split_data() function

In [23]:
# Load the datasets with torchvision.datasets.ImageFolder object

#use print to check that split_data() works?
#call the function
data_path = split_data("/content/flowers")

#for each flower directory
for name in os.listdir("/content/flowers"):
  
  #print size train
  print(len(os.listdir(os.path.join(data_path['train'], name))))

  #print size test
  print(len(os.listdir(os.path.join(data_path['test'], name))))

  #print size valid
  print(len(os.listdir(os.path.join(data_path['valid'], name))))

  #print the data that is not selected and remains in the flowers directory,
  # the size of which should be 0
  print(len(os.listdir(os.path.join("/content/flowers", name))))

#raining_imagefolder = datasets.ImageFolder(os.path.join("/content", data_path['train']), transform = training_transforms)
#validation_imagefolder = datasets.ImageFolder(os.path.join("/content", data_path['valid']), transform = validation_transforms)
#testing_imagefolder = datasets.ImageFolder(os.path.join("/content", data_path['test']), transform = testing_transforms)

535
41
23
165
549
42
24
169
736
57
32
227
689
53
30
212
513
40
22
158


In [ ]:
tr_batchsize = 32
eval_test_batchsize = 16
epochs = 10
lr = 0.001

In [ ]:
#Define the torch.utils.data.DataLoader() object with the ImageFolder object
# Dataloader is a generator to read from ImageFolder and generate them into batch-by-batch
# Only shuffle during trianing, validation and testing no shuffles
# the batchsize for training and tesitng no need to be the same

train_loader = DataLoader(training_imagefolder, batch_size = tr_batchsize, shuffle = True)
validate_loader = DataLoader(validation_imagefolder, batch_size = eval_test_batchsize)
test_loader = DataLoader(testing_imagefolder, batch_size = eval_test_batchsize)

In [ ]:
#Classes name to class index number Mapping with JSON
import json
with open('flower_to_name.json', 'r') as f:
    flower_to_name = json.load(f)

#print(len(flower_to_name))
#print(flower_to_name)

In [ ]:
#load pretrained squeezenet model
model = models.squeezenet1_0(pretrained=True)

In [ ]:
#Freeze the params in the feature head, by setting .requries.grad to False
for params in model.parameters():
    params.requries_grad = False

In [ ]:
#Build a customed classifier head with 102 output classes
#Change the last layer according to the problem

NewClassifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(25088, 4096)),
    ('relu', nn.ReLU()),
    ('drop', nn.Dropout(p = 0.5)),
    ('fc2', nn.Linear(4096, 102)),
    ('output', nn.LogSoftmax(dim = 1))
]))

In [ ]:
#Attach the new classifier head onto the model
model.classifier = NewClassifier

In [ ]:
# Define the loss func and the optimizer

criterion = nn.CrossEntropyLoss() 
# in the optimzier, define the range of params to be updated
# Here we only train the params in the model.classifier part (model object has the classifier attribute)
optimizer = optim.Adam(model.classifier.parameters(), lr = lr)

In [ ]:
#The Validation Function

def validation(model, validateloader, ValCriterion):
    
    val_loss_running = 0
    acc = 0
    
    # a dataloader object is a generator of batches, each batch contain image & label separately
    for images, labels in iter(validateloader):
        
        # Send the data onto choosen device
        images = images.to(deviceFlag)
        labels = labels.to(deviceFlag)
        
        output = model.forward(images)
        val_loss_running += ValCriterion(output, labels).item() # .item() to get a scalar in Torch.tensor out
        
        output = torch.exp(output) # as in the model we use the .LogSoftmax() output layer
        
        equals = (labels.data == output.max(dim = 1)[1])
        acc += equals.float().mean().item() # .flaot() is to transfer the tensor.cuda.float type onto cpu mode
        
    return val_loss_running / len(validateloader), acc / len(validateloader)

In [ ]:
#The Training (&Validating) Function

def train_eval(model, traindataloader, validateloader, TrCriterion, optimizer, epochs, deviceFlag_train):
    
    itrs = 0
    eval_itrs = 40
    
    # first setting the device used for training
    model.to(deviceFlag_train)
    
    print(f'The training batchsize is {tr_batchsize}.')
    
    # set the timer
    since = time.time()

    # ! THE EPOCH LOOP !
    for e in range(epochs):        
        itrs = 0
        
        # Set the model to the Train mode
        # Tell the model to activate its Training behavior (turn-on the dropout & BN behaviors)
        model.train()
        
        # re-initialize the running_loss to start every epoch
        training_loss_running = 0
        
        #  ! THE BATCH LOOP !
        for inputs, labels in iter(traindataloader):            
            itrs += 1
            # .to() method return a copy of the tensors on the targeted device
            inputs = inputs.to(deviceFlag_train)
            labels = labels.to(deviceFlag_train)
            
            # Clean the stored grads computed in the last iteration
            optimizer.zero_grad()
            
            # Forward Pass
            # As model has been shipped to the targeted device, so the output is on that device too
            outputs = model.forward(inputs)
            
            # Compute Loss
            train_loss = TrCriterion(outputs, labels)
            
            # BackProp to compute the grads (stored in each tensor.grad() attributes) along the way
            train_loss.backward()
            
            # Optimizer/Update params
            optimizer.step()
            
            training_loss_running += train_loss.item() #numeric ops, take the scalar out of the tensor by calling .item()
            
            # ----------- Perform Validation (Evaluation) Every eval_itrs iterations ---------- #
            if itrs % eval_itrs == 0:
                
                # Set the model to the Eval mode
                model.eval()
                
                # Turn-off gradient for validation to save memory & computation
                with torch.no_grad():
                    validation_loss, val_acc = validation(model, validateloader, TrCriterion)
                
                display = f'Epoch: {e + 1}/{epochs}, itrs: {itrs}, '
                display += f'Train_loss: {round(training_loss_running / eval_itrs, 4)}, '
                display += f'Valid_loss: {round(validation_loss, 4)}, '
                display += f'Valid_Acc: {round(val_acc, 4)}'
                print(display)
                
                training_loss_running = 0
                model.train()
                
        end = time.time()
        elapsed = end - since
        print(f'Epoch {e + 1} takees {round(elapsed, 4)} sec')

In [ ]:
#The TEST function

def test_acc(model, testloader, deviceFlag_test):

    # for testing, it is actually do validation on the test set
    model.eval()

    model.to(deviceFlag_test)

    since = time.time()

    # In .eval() mode, set the context manager to turn-off grads
    with torch.no_grad():
        acc = 0

        # iter() gives images and labels in batches
        for inputs, labels in iter(test_loader):
            
            inputs = inputs.to(deviceFlag_test)
            labels = labels.to(deviceFlag_test)

            # Do a forward pass
            output = model.forward(inputs)
            # convert the log likelihood to scalar
            prob = torch.exp(output)

            equals = (labels.data == prob.max(dim = 1)[1])

            acc += equals.type(torch.FloatTensor).mean().item()

        end = time.time()
        elapsed = end - since

        print(f'Test_acc: {round(acc, 4)}, tiem_spent: {round(elapsed, 2)} sec')

In [ ]:
def image_preprocessing(img_pth):
    '''
    Input a PIL image, output a numpy array with axes transposed to [Ch, H, W]
    '''
    pil_image = Image.open(img_pth)
    
    # -------- Resize with Aspect Ratio maintained--------- #
    # First fixing the short axes
    if pil_image.size[0] > pil_image.size[1]:
        pil_image.thumbnail((10000000, 256))
    else:
        pil_image.thumbnail((256, 100000000))
    
    # ---------Crop----------- #
    left_margin = (pil_image.width - 224) / 2
    bottom_margin = (pil_image.height - 224) / 2
    right_margin = left_margin + 224
    top_margin = bottom_margin + 224
    
    pil_image = pil_image.crop((left_margin, bottom_margin, right_margin, top_margin))
    
    # --------- Convert to np then Normalize ----------- #
    np_image = np.array(pil_image) / 255
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    np_image = (np_image -mean) / std
    
    # --------- Transpose to fit PyTorch Axes ----------#
    np_image = np_image.transpose([2, 0, 1])
    
    return np_image

In [ ]:
#Image Dispalying Function with Matplotlib

def imshow(pt_image, ax = None, title = None):
    '''
    Takes in a PyTorch-compatible image with [Ch, H, W],
    Convert it back to [H, W, Ch], 
    Undo the preprocessing,
    then display it on a grid
    '''
    if ax is None:
        fig, ax = plt.subplots()
    
    # --------- Transpose ----------- #
    plt_image = pt_image.transpose((1, 2, 0))
    
    # --------- Undo the preprocessing --------- #
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    plt_image = plt_image * std + mean
    
    if title is not None:
        ax.set_title(title)
        
    # Image need to be clipped between 0 and 1 or it looks noisy
    plt_image = np.clip(plt_image, 0, 1)
    
    # this imshow is a function defined in the plt module
    ax.imshow(plt_image)
    
    return ax

In [ ]:
#Prediction Function

def predict(img_pth, model, trainingdataset, topk):
    '''
    1. input a single img;
    2. forward pass on a model;
    3. use tensor.topk(k) to return the highest k probs and the correspodniung class idx;
    4. convert the idx to class names using the name_to_idx conversion.
    '''
    np_img = image_preprocessing(img_pth)
    
    # Convert np_img to PT tensor and send to GPU
    pt_img = torch.from_numpy(np_img).type(torch.cuda.FloatTensor)
    
    # Unsqueeze to get shape of tensor from [Ch, H, W] to [Batch, Ch, H, W]
    pt_img = pt_img.unsqueeze(0)

    # Run the model to predict
    output = model.forward(pt_img)
    
    probs = torch.exp(output)
    
    # Pick out the topk from all classes 
    top_probs, top_indices = probs.topk(topk)
    
    # Convert to list on CPU without grads
    top_probs = top_probs.detach().type(torch.FloatTensor).numpy().tolist()[0]
    top_indices = top_indices.detach().type(torch.FloatTensor).numpy().tolist()[0]
    
    # Invert the class_to_idx dict to a idx_to_class dict
    idx_to_class = {value: key for key, value in trainingdataset.class_to_idx.items()}
    top_classname = {idx_to_class[index] for index in top_indices}
    
    return top_probs, top_classname   

In [ ]:
#Train & Eval the model
train_eval(model, train_loader, validate_loader, criterion, optimizer, epochs, deviceFlag)

In [ ]:
#Test the model
test_acc(model, test_loader, deviceFlag)

In [ ]:
image = image_preprocessing('flowers/test/1/image_06743.jpg')
imshow(image)

In [ ]:
probs, classes = predict('flowers/test/15/image_06369.jpg', model, training_imagefolder, 5)   
print(probs)
print(classes)